In [ ]:
import pandas as pd
import os
import numpy as np

## RQ2

In [ ]:
base_folder_path = os.getcwd().split(os.sep + 'src')[0]
dataset_path = os.path.join(base_folder_path, 'dataset')
print("Dataset Path: " + dataset_path)

xlsx_file_path = dataset_path + os.sep + "combined-temp.xlsx"
result_file_path = dataset_path + os.sep + "combined_analysis.xlsx"

In [ ]:
def find_response_type_dict(df:pd.DataFrame, counter):
    tag_dict = dict()
    explanations = ["EC", "EIC", "EPC"]
    answers = ['AC', 'AIC', 'APC']
    for tag_1 in explanations:
        for tag_2 in answers:
            tag_dict[ tag_1 + '-' + tag_2] = 0

    for index, row in df.iterrows():
        if row['Explanation.'+str(counter)] not in explanations:
            continue
        if row['Answer.'+str(counter)] not in answers:
            continue

        key = row['Explanation.'+str(counter)] + '-' + row['Answer.'+str(counter)]

        if key in tag_dict:
            tag_dict[key] += 1

    return tag_dict

In [ ]:
sheets = pd.ExcelFile(os.path.join(xlsx_file_path)).sheet_names
total_iterations = 3
for sheet in sheets:
    df = pd.read_excel(xlsx_file_path, sheet)
    print( "Sheet:", sheet)

    result = pd.DataFrame()
    for index in range(1, (total_iterations*2)+1):
        tag_dict = find_response_type_dict(df, index)
        result = result.append( pd.DataFrame( [tag_dict]) )

    index_column = pd.Index(['shared_1', 'shared_2', 'shared_3', 'separate_1', 'separate_2', 'separate_3'])
    result = result.set_index(index_column)
    print(result)
    # with pd.ExcelWriter(result_file_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    #     result.to_excel(writer, sheet_name=sheet+"_testing", index=True)

## RQ4: Confidence analysis

In [ ]:
def find_confidence_dict(df:pd.DataFrame):
    tag_dict = dict()
    tags = ["EC", "EIC", "EPC"]
    # tags = ['AC', 'AIC', 'APC']
    for tag in tags:
        tag_dict[ tag ] = dict()

    for index, row in df.iterrows():

        if pd.isna(row['Confidence.2']):
            key = row['Explanation.1']
            if key in tag_dict:
                if row['Confidence.1'] not in tag_dict[key]:
                    tag_dict[key][row['Confidence.1']] = 1
                else:
                    tag_dict[key][row['Confidence.1']] += 1
        else:
            key = row['Explanation.4']
            if key in tag_dict:
                if row['Confidence.2'] not in tag_dict[key]:
                    tag_dict[key][row['Confidence.2']] = 1
                else:
                    tag_dict[key][row['Confidence.2']] += 1

    return tag_dict

In [ ]:
sheets = pd.ExcelFile(os.path.join(xlsx_file_path)).sheet_names

for sheet in sheets:
    df = pd.read_excel(xlsx_file_path, sheet)
    print( "Sheet:", sheet)

    tag_dict = find_confidence_dict(df)
    result = pd.DataFrame( tag_dict)
    result = result. replace(np.nan,0)

    print(result)

    # with pd.ExcelWriter(result_file_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    #     result.to_excel(writer, sheet_name=sheet+"_confidence", index=True)


## RQ1

In [ ]:
def find_tag_dict(df:pd.DataFrame, counter):

    # tags = ["EC", "EIC", "EPC"]
    tags = ["AC", "AIC", "APC"]
    tag_dict = dict()
    for tag in tags:
        tag_dict[tag] = 0

    for index, row in df.iterrows():
        if row['Answer.'+str(counter)] not in tags:
            continue

        tag_dict[row['Answer.'+str(counter)]] += 1

    return tag_dict

In [ ]:
xlsx_file_path = dataset_path + os.sep + "combined_pair.xlsx"
sheets = pd.ExcelFile(os.path.join(xlsx_file_path)).sheet_names
total_iterations = 3

for sheet in sheets:
    df = pd.read_excel(xlsx_file_path, sheet)
    print( "Sheet:", sheet)

    result = pd.DataFrame()
    for index in range(1, (total_iterations*2)+1):
        tag_dict = find_tag_dict(df, index)
        result = result.append( pd.DataFrame( [tag_dict]) )

    index_column = pd.Index(['shared_1', 'shared_2', 'shared_3', 'separate_1', 'separate_2', 'separate_3'])
    result = result.set_index(index_column)
    print(result)
    # with pd.ExcelWriter(result_file_path, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    #     result.to_excel(writer, sheet_name=sheet+"_testing", index=True)

## RQ3

In [ ]:
xlsx_file_path = dataset_path + os.sep + "combined-temp.xlsx"
df = pd.read_excel(xlsx_file_path, 'all')

In [ ]:
consistent = 0
inconsistent = 0

for index, row in df.iterrows():
    prev = row['Explanation.1']

    if pd.isna(prev):
        continue

    flag = False

    for iter in range(2, 4):
        col_name = row['Explanation.'+str(iter)]
        if prev is not col_name:
            inconsistent += 1
            flag = True
            break

    if flag == False:
        consistent += 1

print("Explanation", "Inconsistent:", inconsistent, "Consistent:", consistent)

In [ ]:
consistent = 0
inconsistent = 0

for index, row in df.iterrows():
    prev = row['Answer.1']
    if pd.isna(prev):
        continue

    flag = False

    for iter in range(2, 4):
        col_name = row['Answer.'+str(iter)]
        if prev is not col_name:
            inconsistent += 1
            flag = True
            break

    if flag == False:
        consistent += 1

print("Answer", "Inconsistent:", inconsistent, "Consistent:", consistent)